In [0]:
dbutils.fs.put('dbfs:/FileStore/spark_RealTimeScenarioData/complex_json.json',"""[{
	"id": "0001",
	"type": "donut",
	"name": "Cake",
	"ppu": 0.55,
	"batters":
		{
			"batter":
				[
					{ "id": "1001", "type": "Regular" },
					{ "id": "1002", "type": "Chocolate" },
					{ "id": "1003", "type": "Blueberry" },
					{ "id": "1004", "type": "Devil's Food" }
				]
		},
	"topping":
		[
			{ "id": "5001", "type": "None" },
			{ "id": "5002", "type": "Glazed" },
			{ "id": "5005", "type": "Sugar" },
			{ "id": "5007", "type": "Powdered Sugar" },
			{ "id": "5006", "type": "Chocolate with Sprinkles" },
			{ "id": "5003", "type": "Chocolate" },
			{ "id": "5004", "type": "Maple" }
		]
}]""", True)

Wrote 608 bytes.
Out[1]: True

In [0]:
df = spark.read.json(path='dbfs:/FileStore/spark_RealTimeScenarioData/complex_json.json')
display(df)

_corrupt_record,id,type
[{,null,null
"""id"": ""0001"",",null,null
"""type"": ""donut"",",null,null
"""name"": ""Cake"",",null,null
"""ppu"": 0.55,",null,null
"""batters"":",null,null
{,null,null
"""batter"":",null,null
[,null,null
null,1001,Regular


In [0]:
# reading multiline json data
df = spark.read.json(path='dbfs:/FileStore/spark_RealTimeScenarioData/complex_json.json', multiLine=True)
display(df)

batters,id,name,ppu,topping,type
"List(List(List(1001, Regular), List(1002, Chocolate), List(1003, Blueberry), List(1004, Devil's Food)))",0001,Cake,0.55,"List(List(5001, None), List(5002, Glazed), List(5005, Sugar), List(5007, Powdered Sugar), List(5006, Chocolate with Sprinkles), List(5003, Chocolate), List(5004, Maple))",donut


In [0]:
from pyspark.sql.functions import explode, col

In [0]:
df.printSchema()

root
 |-- batters: struct (nullable = true)
 |    |-- batter: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- ppu: double (nullable = true)
 |-- topping: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- type: string (nullable = true)
 |-- type: string (nullable = true)



In [0]:
df_final = df.withColumn('topping_explode', explode('topping'))\
            .withColumn('topping_id', col('topping_explode.id'))\
            .withColumn('topping_type', col('topping_explode.type'))\
            .drop('topping', 'topping_explode')\
            .withColumn('batter_explode', explode('batters.batter'))\
            .withColumn('batter_id', col('batter_explode.id'))\
            .withColumn('batter_type', col('batter_explode.type'))\
            .drop('batter_explode', 'batters')
display(df_final)

id,name,ppu,type,topping_id,topping_type,batter_id,batter_type
0001,Cake,0.55,donut,5001,None,1001,Regular
0001,Cake,0.55,donut,5001,None,1002,Chocolate
0001,Cake,0.55,donut,5001,None,1003,Blueberry
0001,Cake,0.55,donut,5001,None,1004,Devil's Food
0001,Cake,0.55,donut,5002,Glazed,1001,Regular
0001,Cake,0.55,donut,5002,Glazed,1002,Chocolate
0001,Cake,0.55,donut,5002,Glazed,1003,Blueberry
0001,Cake,0.55,donut,5002,Glazed,1004,Devil's Food
0001,Cake,0.55,donut,5005,Sugar,1001,Regular
0001,Cake,0.55,donut,5005,Sugar,1002,Chocolate
